In [ ]:
import torch
from torchvision import datasets, transforms
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torch.utils.data import DataLoader
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

In [ ]:
# Define the Faster R-CNN architecture
model = fasterrcnn_resnet50_fpn(pretrained=True)

# Modify the roi_heads module to return 2 values
in_features = model.roi_heads.box_predictor.cls_score.in_features
num_classes = 10  # MNIST has 10 classes
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes + 1)  # +1 for the background class

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth
100%|██████████| 160M/160M [00:01<00:00, 137MB/s]


In [ ]:
# Preprocess the MNIST dataset
transform = transforms.Compose([
    transforms.Resize((300, 300)),
    transforms.ToTensor(),
])

trainset = datasets.MNIST(root='C:/Users/hp/Downloads/archive (1)', train=True, download=True, transform=transform)
testset = datasets.MNIST(root='C:/Users/hp/Downloads/archive (1)', train=False, download=True, transform=transform)

trainloader = DataLoader(trainset, batch_size=4, shuffle=True)
testloader = DataLoader(testset, batch_size=4, shuffle=False)


100%|██████████| 9912422/9912422 [00:00<00:00, 162878487.65it/s]

Extracting C:/Users/hp/Downloads/archive (1)/MNIST/raw/train-images-idx3-ubyte.gz to C:/Users/hp/Downloads/archive (1)/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 96908555.06it/s]

Extracting C:/Users/hp/Downloads/archive (1)/MNIST/raw/train-labels-idx1-ubyte.gz to C:/Users/hp/Downloads/archive (1)/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 46227057.53it/s]


Extracting C:/Users/hp/Downloads/archive (1)/MNIST/raw/t10k-images-idx3-ubyte.gz to C:/Users/hp/Downloads/archive (1)/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 5781647.58it/s]

Extracting C:/Users/hp/Downloads/archive (1)/MNIST/raw/t10k-labels-idx1-ubyte.gz to C:/Users/hp/Downloads/archive (1)/MNIST/raw



In [ ]:
# Train the Faster R-CNN model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = torch.optim.SGD(model.parameters(), lr=0.005, momentum=0.9)
num_epochs = 5

for epoch in range(num_epochs):
    model.train()
    for images, targets in trainloader:
        images = [img.to(device) for img in images]

        targets_list = []
        for target in targets:
            if isinstance(target, dict) and "boxes" in target and "labels" in target:
                target_dict = {
                    "boxes": target["boxes"].to(device),
                    "labels": target["labels"].to(device)
                }
                targets_list.append(target_dict)

        if len(targets_list) > 0:
            optimizer.zero_grad()
            loss_dict = model(images, targets_list)
            losses = sum(loss for loss in loss_dict.values())
            losses.backward()
            optimizer.step()

    print(f"Epoch {epoch + 1}/{num_epochs} complete.")

print("Classifier training complete.")

Epoch 1/5 complete.
Epoch 2/5 complete.
Epoch 3/5 complete.
Epoch 4/5 complete.
Epoch 5/5 complete.
Classifier training complete.


In [ ]:
model.eval()

all_labels = []
all_predictions = []

with torch.no_grad():
    for images, targets in testloader:
        images = [img.to(device) for img in images]

        outputs = model(images)

        # Assume targets are tensors
        for output, target in zip(outputs, targets):
            if target.ndim > 0:
                _, predicted = torch.max(output["labels"], -1)
                all_labels.extend(target.cpu().numpy())
                all_predictions.extend(predicted.cpu().numpy())


# Calculate accuracy
correct = sum(1 for true, pred in zip(all_labels, all_predictions) if true == pred)
total = len(all_labels)
accuracy = correct / total
print(f'Accuracy on test set: {accuracy * 100:.2f}%')